In [1]:
import time
import pickle
import os
import re
import csv
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import pysnooper
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from collections import defaultdict
from selenium.webdriver.support import expected_conditions as EC
class KeysManager:
    def __init__(self):
        self.keys,self.temp = self.get_keys()
        self.wait_craw_key = self.keys - self.temp
    def get_keys(self):
        with open('./拓展关键词组合.txt',encoding='utf-8') as f:
            keys = set(f.read().split())
        if os.path.exists('./temp.txt'):
            with open('./temp.txt',encoding='utf-8') as f:
                temp = set(f.read().split())
        else:
            temp = set()
        return keys,temp
    def save_key(self,key):
        with open('./temp.txt','a+',encoding='utf-8') as f:
            f.write(key+' ')
    def get_key(self):
        key = self.wait_craw_key.pop().strip()
        return key


class Crawler:
    def __init__(self):
        self.cookies = None
        self.sign_in_web = "https://www.lagou.com/landing-page/pc/search.html?utm_source=m_cf_cpt_baidu_pcbt"
        self.driver_config()
        self.key_manager = KeysManager()
        self.save_path='./work_data'
    def driver_config(self):
        '''
        用于设置反反爬配置
        :return:
        '''
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument('--window-size=1920x1080')
        chrome_options.add_argument(
            'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36')
        self.webdriver = Chrome(options=chrome_options)
        with open('./stealth.min.js') as f:
            js = f.read()

        self.webdriver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": js
        })


    def sign_in(self):
        '''
        登录，并保存cookies
        :return:
        '''
        self.webdriver.get(self.sign_in_web)
        self.webdriver.find_element_by_class_name("login-by-account").click()
        time.sleep(1)
        windows = self.webdriver.window_handles
        self.webdriver.close()
        self.webdriver.switch_to.window(windows[1])
        self.webdriver.find_element_by_xpath("//input[@placeholder='请输入常用手机号/邮箱']").send_keys('账号')
        self.webdriver.find_element_by_xpath("//input[@placeholder='请输入密码']").send_keys('密码')
        self.webdriver.find_element_by_class_name("no-checked").click()
        self.webdriver.find_element_by_xpath("//input[@type='submit']").click()
        self.save_cookies()
    def save_cookies(self):
        '''
        存储cookies
        :param driver:
        :return:
        '''
        tb_cookies = self.webdriver.get_cookies()
        cookies = {}
        for item in tb_cookies:
            cookies[item['name']] = item['value']
        output_path = open('.\cookies.pickle', 'wb')
        pickle.dump(cookies, output_path)
        output_path.close()

    def load_cookies(self):
        '''
        加载cookies
        :return:
        '''
        self.webdriver.get("https://www.lagou.com")
        if os.path.exists('cookies.pickle'):
            read_path = open('cookies.pickle', 'rb')
            tb_cookies = pickle.load(read_path)
            read_path.close()
        for cookie in tb_cookies:
            self.webdriver.add_cookie({
                "domain": '.lagou.com',
                "name": cookie,
                "value": tb_cookies[cookie],
            })

    @pysnooper.snoop(prefix="get_jobs: ")
    def get_jobs(self):
        '''
        获取搜索页中所有工作
        :return:
        '''
        #windows = self.webdriver.window_handles
        #self.webdriver.switch_to.window(windows[1])
        jobs_abstract = []
        time.sleep(5)
        position_list = self.webdriver.find_elements_by_class_name("list_item_top")
        for i in range(len(position_list)):
            job_info = self.webdriver.find_elements_by_class_name("list_item_top")[i]
            position = job_info.find_element_by_class_name("position")
            position = position.text.split("\n")
            name = position[0].strip()
            if len(name)>10:
                name = name[:10]
            location = re.search(r'[^\W]+', position[1]).group(0)
            other_info = position[3].split(" ")
            salary = other_info[0].strip()
            experience = other_info[1].strip()
            edu_bg = other_info[3].strip()

            try:
                company_ele = job_info.find_element_by_class_name("company")
            
            
                company = company_ele.text.split("\n")
                company_name = company[0].strip()
                other_info = company[1].split(" ")
                industry = other_info[0].strip()
                company_scale = other_info[4].strip()
                jobs_abstract.append([name, salary, experience, edu_bg, location,company_name, industry, company_scale])
            except Exception as e:
                print(e)
                print(company_ele.get_attribute("outerHTML"))
            
        jobs_abstract = DataFrame(np.array(jobs_abstract),columns=["name", "salary", "experience", "edu_bg", "location","company_name", "industry", "company_scale"])
        return  jobs_abstract
    def search(self,key):
        '''
        根据关键字搜索相关工作
        :param key:
        :return:
        '''
        web_path = "https://www.lagou.com/jobs/list_" + key
        #self.load_cookies()
        self.webdriver.get(web_path)
        time.sleep(3)
        self.save_cookies()
    #@pysnooper.snoop(prefix="get_job_describtion: ")
    def get_job_describtion(self):
        '''
        获取每个具体工作的职责和要求
        :return:
        '''
        jobs_describtion = []
        position_page_list = self.webdriver.find_elements_by_class_name("position_link")
        for i in range(len(position_page_list)):
            try:
                
                position_page = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")[i]
                element = position_page.find_element_by_tag_name("h3")
                #position_page.send_keys(Keys.DOWN)
                #
                
                #self.webdriver.execute_script("arguments[0].scrollIntoView();", element)
                #js="var q=document.documentElement.scrollTop=10000"
                #self.webdriver.execute_script(js)
                
                action = ActionChains(self.webdriver).move_to_element_with_offset(element,5,5)
                action.send_keys(Keys.DOWN)
                action.send_keys(Keys.DOWN)
                action.send_keys(Keys.DOWN)
                action.perform()
                time.sleep(2)
                position_page = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")[i]
                element = position_page.find_element_by_tag_name("h3")
                #action.move_to_element_with_offset(element,0,0).click().perform()
                element.click()
                time.sleep(5)
                windows = self.webdriver.window_handles
                self.webdriver.switch_to.window(windows[1])
                WebDriverWait(self.webdriver, 10).until(
                    expected_conditions.presence_of_element_located((By.CLASS_NAME, "job-detail")))
                position_head = self.webdriver.find_element_by_class_name("position-head-wrap-name")
                position_name = position_head.find_element_by_class_name("position-head-wrap-position-name").text.strip()
                if len(position_name) > 10:
                    position_name = position_name[:10]
                position_salary = position_head.find_element_by_class_name("salary").text.split('·')[0].strip()
                position_describtion = self.webdriver.find_element_by_class_name("job-detail").text
                self.webdriver.close()
                self.webdriver.switch_to.window(windows[0])
                position_describtion = position_describtion.split("\n")
                responsibility = []
                demand = []
                check_code = 0

                for index, text in enumerate(position_describtion):
                    if check_code == 0 and re.match(r"1|-|•", text):
                        check_code = 1
                    if check_code == 1:
                        if re.match(r"\d|-|•", text):
                            responsibility.append(text)
                        else:
                            check_code = 2
                    if check_code == 2 and re.match(r"\d|-|•", text):
                        demand.append(text)
                #如果上述筛选方式，无法拆分工作描述，则将其放到responsibility中
                if check_code == 0:
                    responsibility = position_describtion
                jobs_describtion.append([position_name,position_salary,responsibility,demand])
            except:
                jobs_describtion.append([position_name,position_salary["null"],["null"]])
        jobs_describtion = DataFrame(np.array(jobs_describtion),columns=["name","salary","responsibility","demand"])
        return jobs_describtion
    def get_next_page(self):
        '''
        进入下一页
        :return:
        '''
        time.sleep(1)
        #element = self.webdriver.find_element_by_class_name("pager_next")      
        #ActionChains(self.webdriver).move_to_element_with_offset(element,5,5).click().perform()
#         action.send_keys(Keys.DOWN)
#         action.send_keys(Keys.DOWN)
#         action.send_keys(Keys.DOWN)
#         action.perform()
#         time.sleep(2)
        
        self.webdriver.find_element_by_class_name("pager_next").click()
    def save_jobs_info(self,key,jobs_abstract,jobs_describtion):
        '''
        将爬取到的数据存入csv中
        :param key:
        :param jobs_abstract:
        :param jobs_describtion:
        :return:
        '''
        jobs_info = pd.merge(jobs_abstract,jobs_describtion,on=['name','salary'])
        jobs_info.drop_duplicates(list(jobs_info.columns)[:8],inplace=True)
        jobs_info['key'] = key
        file_path = os.path.join(self.save_path,'%s.csv'%key)
        if not os.path.exists(file_path):
            # ['name', 'salary', 'experience', 'edu_bg',
            #         'location','company_name', 'industry', 'company_scale','responsibility','demand','key']
            head = jobs_info.columns
            with open(file_path, 'w+',encoding='utf-8',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(head)
        with open(file_path,'a+',encoding='utf-8',newline='') as f:
            writer = csv.writer(f)
            for index in range(len(jobs_info)):
                writer.writerow(jobs_info.iloc[index,:].values)

    @pysnooper.snoop(prefix="crawer: ")
    def crawer(self,key,sign_in=False,job_num=50):
        if sign_in==False:
            self.sign_in()
            x=input()
        #self.load_cookies()
        self.search(key)
        job_count = 0
        for i in range(30):
            try:
                if job_count<job_num:
                    jobs_abstract = self.get_jobs()
                    jobs_describtion = self.get_job_describtion()
                    self.save_jobs_info(key,jobs_abstract,jobs_describtion)
                    job_count += len(jobs_abstract)
                    try:
                        self.get_next_page()
                    except Exception as e:
                        print(e)
                        break
                    self.save_cookies()
            except Exception as e:
                time_now = time.strftime('[%Y-%m-%d %H:%M:%S]',time.localtime(time.time()))
                print('info: '+str(time_now)+' '+str(key)+' '+str(e))
                break
        #检测csv文件中有多少个
        with open("%s/%s.csv"%(self.save_path,key),encoding='utf-8') as f:
            csv_length = len(f.readlines())
        #判断数据是否被保存
        if csv_length <=1:
            os.remove("%s/%s.csv"%(self.save_path,key))
        else:
            self.key_manager.save_key(key)

def main():
    crawler = Crawler()
    key = crawler.key_manager.get_key()
    crawler.crawer(key, True, job_num=50)
    while(crawler.key_manager.wait_craw_key):
        key = crawler.key_manager.get_key()
        crawler.crawer(key, True, job_num=50)
if __name__ == '__main__':
    #main()
    pass


In [1]:
crawler = Crawler()
crawler.sign_in()
time.sleep(10)
crawler.search('数据分析')
a = crawler.webdriver.find_elements_by_xpath("//a[@class='position_link']")
crawler.get_next_page()
b = crawler.webdriver.find_elements_by_xpath("//a[@class='position_link']")
print(a[0].get_attribute('href'))
a==b

NameError: name 'Crawler' is not defined

# 重构代码
- 1.登录界面后，搜索关键词，把所有的positionid提取出来
- 2.使用requests去访问所有positionid的网址，使用BeautifulSoup解析网页
- 3.使用ip代理，识别Ip被禁的情况，并更换Ip

In [ ]:
import json
import urllib3
from bs4 import BeautifulSoup
from urllib3.exceptions import ProxyError,MaxRetryError
import requests
class Crawler_for_positionid(Crawler):
    def __init__(self):
        super().__init__()
    def get_positionid(self,key,job_num):
        self.search(key)
        positionid_list=[]
        try:
            for i in range(30):
                target = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")
                print(len(target))
                for j in range(len(target)):

                    href = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")[j].get_attribute("href")
                    positionid = re.search(r'\d+',href).group(0)
                    positionid_list.append(positionid)
                check = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")[0].get_attribute("href")
                self.get_next_page()
                time.sleep(2)
                new_check = self.webdriver.find_elements_by_xpath("//a[@class='position_link']")[0].get_attribute("href")
                if check==new_check or len(positionid_list)>job_num:
                    break
        except Exception as e:
            print(key+": ")
            print(e)
            
        return positionid_list
    @classmethod
    def html_from_positionid(self,ip,positionid):
        url = "https://www.lagou.com/wn/jobs/"+positionid+".html"
        headers = {'User-agent':"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.81 Safari/537.36 SE 2.X MetaSr 1.0"}
        http = urllib3.ProxyManager(ip,timeout=5,retries=4)
        #http = urllib3. PoolManager()
        html = http.request('GET',url,headers=headers).data
        return html
    @classmethod
    def get_html_info(self,html):
        bsobj = BeautifulSoup(html)
        position_name = bsobj.find('span',{'class':"position-head-wrap-position-name"}).get_text()
        salary = bsobj.find('span',{'class':"salary"}).get_text()
        other_info = bsobj.find('dd',{'class':"job_request"}).find('h3').get_text().split('/')
        location = other_info[0].strip()
        experience = other_info[1].strip()
        edu_bg = other_info[2].strip()
        company_name =  bsobj.find('div',{'class':"job_company_content"}).get_text()
        industry = bsobj.findAll('h4',{'class':"c_feature_name"})[0].get_text()
        company_scale = bsobj.findAll('h4',{'class':"c_feature_name"})[2].get_text()
        position_describtion = bsobj.find('div',{'class':"job-detail"}).get_text()
        
        position_describtion = position_describtion.split("\n")
        responsibility = []
        demand = []
        check_code = 0
        for index, text in enumerate(position_describtion):
            if check_code == 0 and re.match(r"1|-|•", text):
                check_code = 1
            if check_code == 1:
                if re.match(r"\d|-|•", text):
                    responsibility.append(text)
                else:
                    check_code = 2
            if check_code == 2 and re.match(r"\d|-|•", text):
                demand.append(text)
        #如果上述筛选方式，无法拆分工作描述，则将其放到responsibility中
        if check_code == 0:
            responsibility = position_describtion
        return [position_name,salary,location,experience,edu_bg,company_name,industry,company_scale,responsibility,demand]
        
def get_positionids():
    key_id = defaultdict(list)
    crawler = Crawler_for_positionid()
    crawler.sign_in()
    time.sleep(10)
    key = crawler.key_manager.get_key()
    id_list = crawler.get_positionid(key,job_num=10000)
    key_id[key] = id_list
    crawler.key_manager.save_key(key)
    while(crawler.key_manager.wait_craw_key):
        try:
            key = crawler.key_manager.get_key()
            id_list = crawler.get_positionid(key,job_num=10000)
            key_id[key]=id_list
            crawler.key_manager.save_key(key)
        except Exception as e:
            print(e)
            with open('./key_id.json','r') as f:
                a = json.load(f)
            for key,value in key_id.items():
                a[key] = value
            with open('./key_id.json','w') as f:
                json.dump(a,f)
            WebDriverWait(crawler.webdriver,60).until(EC.presence_of_element_located((By.CLASS_NAME,"position_link")))
    return key_id
def get_ip():
    response = requests.get("http://proxy.httpdaili.com/apinew.asp?sl=3&noinfo=true&text=1&ddbh=1448402233942353385")
    return response.text.split()
#@pysnooper.snoop(prefix="get_info: ")
def get_info(ki_file,destination_file):
    #提取待爬id
    info_dict = defaultdict(dict)
    ip = 'http://'+get_ip()[-1]
    ip_index = -1
    with open(ki_file,'r') as f:
        key_id = json.load(f)
    with open(destination_file,'r') as f:
        info_dict = json.load(f)
        data = DataFrame(info_dict).T
        count_group = data.groupby(data['key'])
    with open('./temp.txt',encoding='utf-8') as f:
        pass_key = f.read().split()
    #pass_key = ['股票交易','数据仓库','数据库系统','需求调研']
    for key,value in count_group: 
        if key in pass_key:
            continue
        if len(key_id[key])>value.shape[0]:
            ids = list(set(key_id[key]) - set(value['id'].values))
            print(len(ids))
            for id_ in ids:
                try:
                    if id_ in info_dict.keys():
                        continue
                        print('id_已经出现过了')
                    html = Crawler_for_positionid.html_from_positionid(ip,id_)
                    info = Crawler_for_positionid.get_html_info(html)
                    info_dict[id_]=\
                    {
                        'id':id_,
                        'key':key,
                        'name':info[0],
                        'salary':info[1],
                        'experience':info[3],
                        'edu_bg':info[4],
                        'location':info[2],
                        'company_name':info[5],
                        'industry':info[6],
                        'company_scale':info[7],
                        'responsibility':info[8],
                        'demand':info[9]
                    }                 
                    print(key,info_dict[id_]['id'])
                    #time.sleep(0.5)
                except (ProxyError,IndexError,AttributeError,MaxRetryError) as e:
                    ip_index = (ip_index+1)%3
                    ip = 'http://'+get_ip()[ip_index]
                    print(e)
                    with open(destination_file,'w') as f:
                        json.dump(info_dict,f)
                    continue            
        with open('./temp.txt','a+',encoding='utf-8') as f:
            f.write(key+' ')
    new_keys = list(set(key_id.keys())-set(data['key'].unique()))
    for key in new_keys:
        for id_ in key_id[key]:
            try:
                if id_ in info_dict.keys():
                    continue
                    print('id_已经出现过了')
                html = Crawler_for_positionid.html_from_positionid(ip,id_)
                info = Crawler_for_positionid.get_html_info(html)
                info_dict[id_]=\
                {
                    'id':id_,
                    'key':key,
                    'name':info[0],
                    'salary':info[1],
                    'experience':info[3],
                    'edu_bg':info[4],
                    'location':info[2],
                    'company_name':info[5],
                    'industry':info[6],
                    'company_scale':info[7],
                    'responsibility':info[8],
                    'demand':info[9]
                }
                print(key,info_dict[id_]['id'])
                #time.sleep(0.5)              
            except (ProxyError,IndexError,AttributeError,MaxRetryError) as e:
                ip_index = (ip_index+1)%3
                print(e)
                ip = 'http://'+get_ip()[ip_index]
                with open(destination_file,'w') as f:
                    json.dump(info_dict,f)
                continue 
    with open(destination_file,'w') as f:
        json.dump(info_dict,f)

    
if __name__ == '__main__':
    #key_id = main()
    pass

In [31]:

get_info('key_id.json','data.json')

77
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
python 8333962
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' objec

HTTPSConnectionPool(host='www.lagou.com', port=443): Max retries exceeded with url: /wn/jobs/6634617.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F2E3BC92B0>, 'Connection to 114.99.252.58 timed out. (connect timeout=5)'))
'NoneType' object has no attribute 'get_text'
HTTPSConnectionPool(host='www.lagou.com', port=443): Max retries exceeded with url: /wn/jobs/9777189.html (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F2E3BC94C0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))
HTTPSConnectionPool(host='www.lagou.com', port=443): Max retries exceeded with url: /wn/jobs/9250382.html (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F2EA6AA3A0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))
'NoneType' object has no attribute 'get_text'
H

LocationParseError: Failed to parse: http://23:21:00

In [120]:
destination_file='data.json'
with open(destination_file,'r') as f:
    info_dict = json.load(f)
    data = DataFrame(info_dict).T


print(len(info_dict.keys()))
print(data.loc['6267662',:])

1207


In [160]:
with open('key_id.json','r') as f:
    key_id = json.load(f)
key_id.keys()

dict_keys(['股票交易', '统计模型', '产品优化', '运营管理', '金融风险', '需求分析', '风险模型', '营销策略', '信贷风险', '风控模型', '数据治理', '产品运营', '报表审计', '欺诈风险', '金融保险', '金融市场', '财务审计', '基金大数据', '数据库', '算法开发', '数据挖掘', '风险分析', '基金产品', '测试开发', '市场运营', '财务欺诈', '电商运营', '营销管理', '产品推广', '数据存储', '软件开发', 'excel', '量化研究', 'linux', 'hadoop', '语音数据', '用户推广', '数据反爬', '产品测试', '统计分析', '互联网运维', '指标体系', '数据库系统', '风险管理', '市场研究', '信贷模型', '基金报表', '模型开发', '财务指标', '金融业务', 'spark', '保险', '量化分析', '电商指标', '财务报表', '评分模型', '统计建模', '供应链大数据', 'sql', '人工智能', '统计研究', '营销分析', '供应链优化', '增长分析', '深度学习', '产品营销', '量化模型', '电商系统', '自然语言处理', '交易模型', '股票指标', '数据采集', '临床试验统计', '风险预测', '算法研究', '计算机视觉', 'pytorch', '电商交易', '大数据建模', '自动驾驶', '算法分析', '财务管理', '量化基金', '统计', '模型验证', '股票报表', 'hive', '电商大数据', '信贷欺诈', '算法测试', '信贷审计', '基金指标', '建模分析', '预测算法', '保险产品', '信贷大数据', '测试框架', '数据整理', '用户需求', '数据科学', '数据产品', '大数据系统', 'python', '编程开发', '量化交易', '互联网数据', '电商营销', '风险识别', '互联网运营', '股票预测', '精算师', '市场调研', '需求调研', '股票量化', '电商产品', '金融', 'java', '语音识别', '医学大数据', '量化指标', '产品调研', '量

In [103]:
with open('./temp.txt','a+',encoding='utf-8') as f:
    for key in a.keys():
        f.write(key+' ')


In [ ]:
# #ip = '124.88.67.81:80'
# ip='http://171.83.189.184:13456'
# html = Crawler_for_positionid.html_from_positionid(ip,'9399072')
# print(html.decode("UTF-8" ))

In [255]:
Crawler_for_positionid.get_html_info(html)

['反欺诈风控系统开发工程师 ',
 '20k-40k·14薪',
 '上海',
 '经验3-5年',
 '本科及以上',
 '阅文集团',
 '内容社区',
 '500-2000人',
 ['工作职责:参与风控系统的开发，负责反欺诈等新技术的研究和应用，结合风控业务具体问题，利用团伙识别、异常检测、聚类分析等构建用户行为反欺诈模风控型；负责风控策略的数据分析，为风控服务提供数据支撑；参与风控技术调研及核心模块的研发；制定、编写相关技术文档。任职资格:熟悉常用的机器学习算法，有数据挖掘项目经验；1年以上风控相关工作经验；熟练掌握Java、Python等编程语言；较强的逻辑思维能力，善于解决和分析有挑战的问题；有实际的风控系统开发经验，有反作弊、反欺诈开发经验；有较强责任感和荣誉感，对自己工作产出负责，既能发挥创造力，又不偏离公司及项目目标；具有良好的理解领悟能力、沟通能力，能够与团队形成良好配合，乐于承担工作压力。'],
 []]

In [5]:
# url = "https://www.lagou.com/wn/jobs/9746367.html"
# #proxy = urllib3.ProxyManager('http://182.34.21.162:13456')
# #proxy = urllib3.PoolManager()
# r1 = proxy.request('GET',"https://www.baidu.com")
# #r1 = proxy.request('GET',"http://httpbin.org/ip")
# r1.data.decode()

In [ ]:
import requests
# proxies = { "http": "http://localhost:9999"}
# response = requests.get("http://httpbin.org/ip",proxies=proxies)
# print(response.text)
response = requests.get("http://proxy.httpdaili.com/apinew.asp?text=true&noinfo=true&sl=10&ddbh=2300692395120353385")
a = response.text.split()
a


In [ ]:
# import requests

# proxy = {
#     'http': '58.55.252.58:8080'
# }

# response = requests.get("http://httpbin.org/ip",proxies=proxy)
# print(response.text)


In [ ]:
# # -*- coding: utf-8 -*-
# import requests

# url = "http://myip.kkcha.com"
# #使用proxies构建一个字典的形式使用
# proxies = {"http": "122.6.226.55:8118"}
# response = requests.get(url=url, proxies=proxies)
# print(response.text)

